In [16]:
from model import MuliLayerTextGCN
import torch
from tqdm import tqdm
import pandas as pd
from model import Vocaburary
# from time import time
# from datetime import timedelta
import os
import pickle

In [17]:
N_layers = [i for i in range(2, 7)]

In [18]:
EPOCH = 300
HIDDEN_DIM = 200

dataset_names = {
    "20NewsGroup": "20NG",
    "MR":"mr",
    "Ohsumed":"ohsumed_single_23",
    "R52":"R52",
    "R8":"R8"
}

result = {k : {} for k in dataset_names.keys()}

SAVE_PATH = './result/multi_layer.result'

In [19]:
if not os.path.isfile(SAVE_PATH):
    for key in dataset_names.keys():
        print("===================================")
        print(key)
        dir_name = dataset_names[key]

        dict_data = torch.load(f'./ProcessedData/{dir_name}/WholeGraphDict.gh')
        voc : Vocaburary = dict_data['voc']
        whole_graph = dict_data['whole_graph'].cuda()
        word_num = dict_data['W']
        label_num = dict_data['L']
        doc_num = dict_data['D']
        train_mask = dict_data['train_mask'].cuda()
        doc_Y : torch.Tensor = dict_data['doc_Y'].cuda()
        word_Y : torch.Tensor = dict_data['word_Y'].T.cuda()
        label_Y : torch.Tensor = dict_data['label_Y'].cuda()
        train_words = list(dict_data['train_word'])
        test_words = list(dict_data['test_word'])
        train_words.sort()
        test_words.sort()
        train_num = train_mask.count_nonzero().cpu().item()
        test_num = doc_num - train_num

        result[key]['statistic'] = {
        "#DOC":doc_num,
        "#Word":word_num,
        "#Class":label_num,
        "#Train" : train_num,
        "#Test" : test_num,
        "#NODE" : word_num + doc_num + label_num
        }
        result[key]["N_ACC"] = []
        result[key]["N_layers"] = N_layers
        for N_layer in result[key]["N_layers"]:
            model = MuliLayerTextGCN(whole_graph.shape[0], HIDDEN_DIM, label_num, N_layer=N_layer).cuda()
            optim = torch.optim.Adam(model.parameters(), lr=1e-3)
            loss_fn = torch.nn.CrossEntropyLoss()
            trainingProcess = tqdm(range(EPOCH))
            test_accs = []
            for epoch in trainingProcess:
                total_loss = 0.
                optim.zero_grad()
                y_hat = model(whole_graph)
                doc_Y_hat = y_hat[:doc_num]
                word_Y_hat = y_hat[doc_num:-label_num]
                label_Y_hat = y_hat[doc_num+word_num :]
                doc_loss = loss_fn(doc_Y_hat[train_mask], doc_Y[train_mask])
                word_loss = loss_fn(word_Y_hat[train_words], word_Y[train_words])
                label_loss = loss_fn(label_Y_hat, label_Y)
                loss = 1.0 * doc_loss + 1.0 * word_loss  + 1.0 * label_loss
                loss.backward()
                optim.step()
                loss_val = loss.item()
                with torch.no_grad():
                    acc_val = ((doc_Y_hat.argmax(1)[~train_mask] == doc_Y.cuda()[~train_mask]).sum() / (~train_mask).sum()).item()
                trainingProcess.set_postfix({"LOSS": loss_val, "ACC" : acc_val})
                test_accs.append(acc_val)
            result[key]['N_ACC'].append(max(test_accs))
    with open(SAVE_PATH, 'wb') as f:
        pickle.dump(result, f)
else:
    with open(SAVE_PATH, 'rb') as f:
        result = pickle.load(f)


In [20]:
processed_data = {}
processed_data['N_layers'] = N_layers
for dataset, data in result.items():
    processed_data[dataset] = list(map(lambda x : x * 100.,data['N_ACC']))

In [21]:
df = pd.DataFrame(processed_data)
df = df.reindex(columns=['N_layers','MR', 'R8', 'R52', 'Ohsumed', '20NewsGroup'])
df = df.rename(columns={"20NewsGroup" : "20NG", "N_layers" : "Layer"})

In [25]:
df.to_csv('./result/multi_layer.csv', index=False)